<a href="https://colab.research.google.com/github/Chri2704/project-in-LangChain/blob/main/Thesis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install --upgrade langchain
!pip install transformers
!pip install openai
!pip install tiktoken
!pip install pypdf docx2txt chromadb
!pip install gradio

import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.document_loaders import PyPDFLoader
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
import sys
import gradio
import warnings
warnings.filterwarnings("ignore")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 23.6 MB/s eta 0

In [ ]:
#upload file

loader = PyPDFLoader("/content/drive/MyDrive/thesis/2305.14552.pdf")
file = loader.load()


#llm_model = "gpt-3.5-turbo"
llm_model = "text-davinci-003"
llm = OpenAI(temperature=0.6,model_name= llm_model, max_tokens=500)



# dividiamo i dati in blocchi di 1.000 caratteri, con una sovrapposizione
# di 200 caratteri tra i blocchi, che aiuta a fornire risultati migliori
# e a contenere il contesto delle informazioni tra i blocchi
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(file)

print(type(documents))

#creiamo vector db prendendo i documenti, usando l'embedding di openai e creando tutto in una dir instruction
vectordb = Chroma.from_documents(
  documents,
  embedding=OpenAIEmbeddings(),
  persist_directory='./instruction'
)
vectordb.persist()



<class 'list'>


In [ ]:
#parte 2
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.prompts.prompt import PromptTemplate
memory = ConversationSummaryBufferMemory(llm=llm,memory_key="chat_history", return_messages=True, max_token_limit= 150)

def wrap_text(text, line_length):
    wrapped_text = ""
    for i in range(0, len(text), line_length):
        wrapped_text += text[i:i+line_length] + '\n'
    return wrapped_text

qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 6}), #search_kwargs={'k': 5} si può impostare in base ai blocchi che desiaderiamo inserire ma aumentandolo si supera facilmente il max token
    #return_source_documents=True,
    memory=memory,
    verbose=True
)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
print(question_generator.prompt.template)
# print(qa_chain.combine_docs_chain.llm_chain.prompt.template)

# Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

# {context}

# Question: {question}
# Helpful Answer:


# chat_history = [] #array per conversazione
# while True:
#     # this prints to the terminal, and waits to accept an input from the user
#     query = input('Prompt: ')
#     # give us a way to exit the script
#     if query == "exit" or query == "quit" or query == "q":
#         print('Exiting')
#         sys.exit()

#     # we pass in the query to the LLM, and print out the response.
#     # altrimenti iene passata la domanda al modello per ottenere risposta
#     result = qa_chain({'question': query})
#     print('Answer: ' + wrap_text(result['answer'],100))
#     # we build up the chat_history list, based on our question and response
#     # from the LLM, and the script then returns to the start of the loop
#     # and is again ready to accept user input.
#     chat_history.append((query, result['answer']))

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


In [ ]:
#parte 2
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

def wrap_text(text, line_length):
    wrapped_text = ""
    for i in range(0, len(text), line_length):
        wrapped_text += text[i:i+line_length] + '\n'
    return wrapped_text


#chain llm e prompt
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
print(CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="map_reduce")

qa_chain = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

chat_history = [] #array per conversazione
while True:
    # this prints to the terminal, and waits to accept an input from the user
    query = input('Prompt: ')
    # give us a way to exit the script
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()

    # we pass in the query to the LLM, and print out the response.
    # altrimenti iene passata la domanda al modello per ottenere risposta
    result = qa_chain({"question": query, "chat_history": chat_history})
    print('Answer: ' + wrap_text(result['answer'],100))


# def get_chat_history(inputs) -> str:
#     res = []
#     for human, ai in inputs:
#         res.append(f"Human:{human}\nAI:{ai}")
#     return "\n".join(res)
# qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectordb.as_retriever(), get_chat_history=get_chat_history)

input_variables=['chat_history', 'question'] template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:'
Prompt: what's document talk about?
Answer:  This paper discusses two major sources of hallucination in language models (LLMs) when asked to per
form natural language inference, and how users of LLMs may be subjected to faulty judgements if the 
content of their request overlaps with data in pretraining. It also examines how LLMs may be able to
 answer questions by recalling information from other sources during pretraining, and how this can l
ead to illogical, contradictory, or misrepresentative answers.

Prompt: what are the conclusions?
Answer:  LLM performance on natural language inference tasks is largely achieved by recall of relevant memor
izations and corpus-based methods.

Prompt: tell me somethi

SystemExit: ignored

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

#chain llm e prompt
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
print(CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="map_reduce")

qa_chain = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

chat_history = []

def answer_question(input_text):
    if input_text.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': input_text, 'chat_history': chat_history})
    chat_history.append((input_text, result['answer']))
    return result['answer']

iface = gradio.Interface(
    fn=answer_question,
    inputs=gradio.Textbox(placeholder="Scrivi qui", label="Domanda"),
    outputs=gradio.Textbox(placeholder="Risposta", label="Risposta"),
    title="Chatbot",
    theme="chat",  # Scegli il tema che preferisci
    layout="vertical",  # Utilizza una disposizione verticale
)

iface.launch()

input_variables=['chat_history', 'question'] template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:'
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b1b7c35f9bf75dfc21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
chat_history = []

def answer_question(input_text, chat_history):
    return result['answer']

iface = gradio.ChatInterface(
    fn=answer_question,
    title="Chatbot",
    theme="compact",  # Scegli il tema che preferisci
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f89c819b38085e9677.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import shutil

# Specifica il percorso della directory da eliminare
directory_da_eliminare = "./instruction"

# Elimina la directory e tutti i suoi contenuti
shutil.rmtree(directory_da_eliminare)

if not os.path.exists(directory_da_eliminare):
    print(f"La directory {directory_da_eliminare} è stata eliminata con successo.")
else:
    print(f"La directory {directory_da_eliminare} non è stata eliminata.")

La directory ./instruction è stata eliminata con successo.
